In [1]:
import pytorch_lightning as pl
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torchvision.datasets import DatasetFolder
from torchvision import transforms
from torch.utils.data import Subset
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import albumentations as alb
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb

In [2]:
project_name='CNN_NATURALIST_'
wandb.login(key="5bfaaa474f16b4400560a3efa1e961104ed54810")
wandb.init(project=project_name)

wandb: Currently logged in as: aravind_iitm. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\PhysioSens\.netrc


In [3]:
###function for loading the image

def load(path):
    img=cv2.imread(path)
    img=cv2.resize(img,(224,224),cv2.INTER_AREA)
    img=img/255
    return img


In [4]:
###path for train and test####

train_path=r'D:\DL_DATA\naturalist\inaturalist_12K\train'
test_path=r'D:\DL_DATA\naturalist\inaturalist_12K\val'


In [5]:
transform=transforms.ToTensor()


In [6]:
###loading the train and test data as torch tensors###
train_data_=DatasetFolder(root=train_path,loader=load,extensions='.jpg',transform=transform)
test_data=DatasetFolder(root=test_path,loader=load,extensions='.jpg',transform=transform)


In [7]:
####taking the indices for train and val so that number of image in each class is same#####
train_indx=[]
val_indx=[]
for i in range(0,len(train_data_),1000):
    train_indx+=list(range(i,i+800))
    val_indx+=list(range(i+800,i+1000))
val_indx=val_indx[0:-1]

In [8]:
#####splitting the 20% of train data into val data
train_data=Subset(dataset=train_data_,indices=train_indx)
val_data=Subset(dataset=train_data_,indices=val_indx)

In [9]:
###### creating data loader for train, val and test
train_loader=DataLoader(train_data,batch_size=32,shuffle=True,drop_last=True)
val_loader=DataLoader(val_data,batch_size=32,shuffle=True,drop_last=True)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True,drop_last=True)

#### using albumentation library for augmentation
from torchvision import transforms
augmentor=transforms.Compose([
    transforms.RandomHorizontalFlip(p=.2),
    transforms.RandomVerticalFlip(p=.2),
    transforms.RandomAutocontrast(p=.2),
    transforms.RandomRotation(degrees=(-10,10)),
    transforms.RandomAdjustSharpness(sharpness_factor=0,p=.2),
    transforms.Resize((224,224))
])

In [16]:
class CNN_NATURALIST(pl.LightningModule):
    def __init__(self,data_aug=False):
        super().__init__()
        self.data_aug=data_aug
        
        
        model=torchvision.models.vgg19(pretrained=True)
        for params in model.features[0:15].parameters():
            params.requires_grad=False
        self.model=model
        self.model.classifier[6]=nn.Linear(4096,10)
        
        self.loss_fun=nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam(self.model.parameters(),lr=1e-4)
        
        self.train_loss=[]
        self.train_acc=[]
        self.val_loss=[]
        self.val_acc=[]
        
    def forward(self,x):
        
        ypred=self.model(x)
        #ypred=nn.Softmax(dim=1)(ypred)
            
        return ypred
    
    def training_step(self,batch,batch_indx):  ####training happens in this function
        img,label=batch
        img=img.float()
        if self.data_aug:
            img=[augmentor(img_) for img_ in img]
            img=torch.stack(img)
                
            
        
        ypred=self(img)
        
        loss=self.loss_fun(ypred,label)
        
        accuracy=(torch.argmax(ypred,dim=1)==label).sum()
        
        accuracy=accuracy/len(ypred)
        
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)   ###calculating train_loss after each batch
        self.log('train_accuracy', accuracy, on_step=False, on_epoch=True, prog_bar=True) ###calculating in the train_accuracy after each batch
        self.train_loss.append(loss.cpu().detach().numpy())
        self.train_acc.append(accuracy.cpu().detach().numpy())
        
        return loss
    
    def on_train_epoch_end(self):
        epoch_loss=np.average(self.train_loss)
        epoch_accuracy=np.average(self.train_acc)
        
        
        self.train_loss.clear()
        self.train_acc.clear()
        
        wandb.log({"Train_Accuracy":np.round(epoch_accuracy*100,2),"Train_Loss":epoch_loss,"Epoch":self.current_epoch})
        ### logging train accuracy and loss in wandb
        
        #print(f"Epoch{self.current_epoch} Final train loss {epoch_loss} and final train accuracy {epoch_accuracy}")
        
    def validation_step(self,batch,batch_indx):
        img,label=batch
        img=img.float()
        
        ypred=self(img)
        
        loss=self.loss_fun(ypred,label)
        
        accuracy=(torch.argmax(ypred,dim=1)==label).sum()
        
        accuracy=accuracy/len(ypred)
        
        self.log('val_loss', loss,prog_bar=True)  ###logging in the val_loss after each batch
        self.log('val_accuracy', accuracy,prog_bar=True) ###logging in the val_accuracy after each batch
        self.val_loss.append(loss.cpu().detach().numpy())
        self.val_acc.append(accuracy.cpu().detach().numpy())
        
        return loss
    
    def on_validation_epoch_end(self):
        epoch_loss=np.average(self.val_loss)
        epoch_accuracy=np.average(self.val_acc)
        
        
        self.val_loss.clear()
        self.val_acc.clear()
        
        wandb.log({"Val_Accuracy":np.round(epoch_accuracy*100,2),"Val_Loss":epoch_loss,"Epoch":self.current_epoch})
        ### logging val accuracy and loss in wandb
        
        #print(f"Epoch{self.current_epoch} Final val loss {epoch_loss} and final val accuracy {epoch_accuracy}")
        
        
            
    def configure_optimizers(self):
        return [self.optimizer]

In [12]:
monitor=ModelCheckpoint(monitor='val_accuracy',save_top_k=1,mode='max')
model=CNN_NATURALIST(data_aug=True)
trainer=pl.Trainer(accelerator='auto',max_epochs=20,callbacks=monitor)
trainer.fit(model,train_loader,val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | VGG              | 139 M 
1 | loss_fun | CrossEntropyLoss | 0     
----------------------------------------------
137 M     Trainable params
1.7 M     Non-trainable params
139 M     Total params
558.445   Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=20` reached.


In [18]:
model=CNN_NATURALIST.load_from_checkpoint(r'C:\Users\PhysioSens\Desktop\python-jupyter\CS6910\CNN\lightning_logs\version_19\checkpoints\epoch=11-step=3000.ckpt')

In [20]:
count=0
for img, label in test_data:
    img=img.unsqueeze(0)
    img=img.float()
    img=img.to('cuda')
    pred=model(img)
    pred=torch.argmax(pred,axis=1)

    if pred==label:
        count+=1
count/len(test_data)

0.7035